In [1]:
import os
os.chdir('/Users/zhanwenxin/Documents/GitHub/cuTAGI-LSTM')

from examples.data_loader import TimeSeriesDataloader, SyntheticTimeSeriesDataloader
import numpy as np
import random

from pytagi.nn import LSTM, Linear, Sequential
from pytagi.hybrid import LSTM_SSM
from tqdm import tqdm
from pytagi import exponential_scheduler
from pytagi.hybrid import process_input_ssm
from pytagi import Normalizer as normalizer
import pytagi.metric as metric

import matplotlib.pyplot as plt

In [2]:
# Dataset
output_col = [0]
num_features = 2
input_seq_len = 26
output_seq_len = 1
seq_stride = 1

ts_mean = np.array([1.0649368, 26.64127424])
ts_std = np.array([1.0880665, 14.98919694])

random_column = random.randint(0, 19)

In [3]:
train_dtl = SyntheticTimeSeriesDataloader(
    # x_file="data/HQ/syn_dataset_hqts2_lstm_unstandardized_20episodes.csv",
    # select_column = 7,
    x_file="data/HQ/syn_dataset_hqts2_lstm_unstandardized_model4_ts1.csv",
    date_time_file="data/HQ/syn_dataset_hqts2_lstm_datetime.csv",
    x_mean=ts_mean,
    x_std=ts_std,
    output_col=output_col,
    input_seq_len=input_seq_len,
    output_seq_len=output_seq_len,
    num_features=num_features,
    stride=seq_stride,
    time_covariates = ['week_of_year'],  # 'hour_of_day','day_of_week', 'week_of_year', 'month_of_year','quarter_of_year'
)

In [4]:
num_epochs: int = 30
batch_size: int = 1
sigma_v: float = 1

# Network
net = Sequential(
    LSTM(num_features, 30, input_seq_len),
    LSTM(30, 30, input_seq_len),
    Linear(30 * input_seq_len, 1),
)
net.set_threads(8)
# #net.to_device("cuda")

phi_AA = 0.997
Sigma_AR = 0.11453**2
phi_AR = 0.51824134
Sigma_AA = Sigma_AR*1e-16
LA_var_stationary = Sigma_AA /(1-phi_AA**2)
AR_var_stationary = Sigma_AR /(1-phi_AR**2)
# Autoregressive acceleration + plain AR
hybrid = LSTM_SSM(
    neural_network = net,           # LSTM
    baseline = 'AA + plain_AR', # 'level', 'trend', 'acceleration', 'ETS'
    zB  = np.array([0.1, 1E-4, 0, 0.02]),    # initial mu for baseline hidden states
    SzB = np.array([1E-5, 1E-8, LA_var_stationary, AR_var_stationary]),    # var
    phi_AA = phi_AA,
    phi_AR = phi_AR,
    Sigma_AR = Sigma_AR,
    Sigma_AA = Sigma_AA,
    use_online_AR=False,
)

# -------------------------------------------------------------------------#
# Training
mses = []

pbar = tqdm(range(num_epochs), desc="Training Progress")
for epoch in pbar:
    batch_iter = train_dtl.create_data_loader(batch_size, shuffle=False)

    # Decaying observation's variance
    sigma_v = exponential_scheduler(
        curr_v=1E-12, min_v=1E-12, decaying_factor=1, curr_iter=epoch
    )
    var_y = np.full((batch_size * len(output_col),), sigma_v**2, dtype=np.float32)

    # Initialize list to save
    hybrid.init_ssm_hs()
    mu_preds_lstm = []
    var_preds_lstm = []
    mu_preds_unnorm = []
    obs_unnorm = []
    mu_ar = []
    var_ar = []
    mu_aa = []
    var_aa = []

    for x, y in batch_iter:
        mu_x, var_x = process_input_ssm(
            mu_x = x, mu_preds_lstm = mu_preds_lstm, var_preds_lstm = var_preds_lstm,
            input_seq_len = input_seq_len, num_features = num_features,
            )

        # Feed forward
        y_pred, _, z_pred, Sz_pred, m_pred, v_pred = hybrid(mu_x, var_x)
        # Backward
        hybrid.backward(mu_obs = y, var_obs = var_y)

        # Training metric
        pred = normalizer.unstandardize(
            y_pred.flatten(), train_dtl.x_mean[output_col], train_dtl.x_std[output_col]
        )
        obs = normalizer.unstandardize(
            y, train_dtl.x_mean[output_col], train_dtl.x_std[output_col]
        )
        mse = metric.mse(pred, obs)
        mses.append(mse)
        mu_preds_lstm.extend(m_pred)
        var_preds_lstm.extend(v_pred)
        obs_unnorm.extend(y)
        mu_preds_unnorm.extend(y_pred)
        mu_ar.append(z_pred[-2].item())
        var_ar.append(Sz_pred[-2][-2])
        mu_aa.append(z_pred[2].item())
        var_aa.append(Sz_pred[2][2])

    # Smoother
    hybrid.smoother()

    mu_smoothed = np.array(hybrid.mu_smoothed)
    cov_smoothed = np.array(hybrid.cov_smoothed)

    # Figures for each epoch
    plt.switch_backend('Agg')
    plt.figure()
    plt.plot(obs_unnorm, color='r',label=r"obs.")
    plt.plot(mu_smoothed[:,0,:],color='k',label=r"level")
    plt.fill_between(np.arange(len(mu_smoothed[:,0,:])), np.array(mu_smoothed[:,0,:]).flatten() - np.sqrt(cov_smoothed[:, 0, 0]), np.array(mu_smoothed[:,0,:]).flatten() + np.sqrt(cov_smoothed[:, 0, 0]), color='k', alpha=0.3, label='_nolegend_')
    plt.plot(mu_smoothed[:,-2,:],color='g',label=r"AR")
    plt.fill_between(np.arange(len(mu_smoothed[:,-2,:])), np.array(mu_smoothed[:,-2,:]).flatten() - np.sqrt(cov_smoothed[:, -2, -2]), np.array(mu_smoothed[:,-2,:]).flatten() + np.sqrt(cov_smoothed[:, -2, -2]), color='g', alpha=0.3, label='_nolegend_')
    plt.plot(mu_preds_unnorm,color='b',label=r"pred.")
    plt.plot(mu_smoothed[:,-1,:],color='orange',label=r"LSTM")
    plt.fill_between(np.arange(len(mu_smoothed[:,-1,:])), np.array(mu_smoothed[:,-1,:]).flatten() - np.sqrt(cov_smoothed[:, -1, -1]), np.array(mu_smoothed[:,-1,:]).flatten() + np.sqrt(cov_smoothed[:, -1, -1]), color='orange', alpha=0.3, label='_nolegend_')
    plt.legend(ncol = 3, loc='upper left')
    filename = f'saved_results/retrain_syn/hybrid_epoch_#{epoch}.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    # figure for AR
    plt.figure(figsize=(20, 2.5))
    plt.plot(np.arange(len(mu_ar)),mu_ar,label=r"AR")
    plt.fill_between(np.arange(len(mu_ar)), np.array(mu_ar) - np.sqrt(var_ar), np.array(mu_ar) + np.sqrt(var_ar), color='gray', alpha=0.3, label='±1 SD')
    plt.ylim(-0.55, 0.55)
    filename = f'saved_results/retrain_syn/phi_AR_epoch_#{epoch}.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    # figure for AA
    plt.figure()
    plt.plot(np.arange(len(mu_aa)),mu_aa,color='b',label=r"AR")
    plt.fill_between(np.arange(len(mu_aa)), np.array(mu_aa) - np.sqrt(var_aa), np.array(mu_aa) + np.sqrt(var_aa), color='blue', alpha=0.3, label='±1 SD')
    filename = f'saved_results/retrain_syn/AA_epoch_#{epoch}.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    # Progress bar
    pbar.set_description(
        f"Epoch {epoch + 1}/{num_epochs}| mse: {np.nanmean(mses):>7.2f}",
        refresh=True,
    )

Epoch 30/30| mse:    0.03: 100%|██████████| 30/30 [03:36<00:00,  7.21s/it]


In [5]:
# Save mu_ar and var_ar in a json file
import json
data = {}
data['mu_ar'] = mu_ar
data['var_ar'] = var_ar

with open('saved_results/retrain_syn/retrain_AR.json', 'w') as outfile:
    json.dump(data, outfile)